In [5]:
from hana_ml import dataframe
from hana_ml.algorithms.pal.neural_network import MLPClassifier, MLPRegressor

from data_load_utils import Settings
from hana_ml.dataframe import create_dataframe_from_pandas

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

# save load_iris() sklearn dataset to iris
# if you'd like to check dataset type use: type(load_iris())
# if you'd like to view list of attributes use: dir(load_iris())
iris = load_iris()

# np.c_ is the numpy concatenate function
# which is used to concat iris['data'] and iris['target'] arrays 
# for pandas column argument: concat iris['feature_names'] list
# and string list (in this case one string); you can make this anything you'd like..  
# the original dataset would probably call this ['Species']
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

url, port, user, pwd = Settings.load_config("./e2edata.ini")
connection_context = dataframe.ConnectionContext(url, port, user, pwd)
hana_df = create_dataframe_from_pandas(connection_context, data, 'IRIS', force=True)

In [6]:
hana_df.head(10).collect()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


In [12]:
from hana_ml.model_storage import ModelStorage
from hana_ml.model_storage_services import ModelSavingServices


model = MLPRegressor(connection_context, hidden_layer_size=[10,], \
                     activation='TANH', output_activation='TANH', learning_rate=0.01, momentum=0.001)
model.fit(hana_df, label='target')

# Creates an object model_storage
MODEL_SCHEMA='STORAGE'
# model storage must use the same connection than the model
model_storage = ModelStorage(connection_context=model.conn_context,
                             schema=MODEL_SCHEMA)

# Saves the model
model.name = 'Model A'  # The model name is mandatory
model.version = 1
model_storage.save_model(model=model)
#need to increase version

# Lists models
model_storage.list_models()


,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,MODEL_STORAGE_VER
0,Model B,1,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""thread_ratio"": null, ""p...",2019-07-29 10:26:18,1
1,Model B,2,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""thread_ratio"": null, ""p...",2019-07-29 10:26:25,1
2,Model A,1,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""thread_ratio"": null, ""p...",2019-07-29 10:26:34,1
3,Model A,2,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""thread_ratio"": null, ""p...",2019-07-29 10:26:42,1
4,Model A,3,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""thread_ratio"": null, ""p...",2019-07-29 10:26:48,1


In [ ]:
model_storage.delete_model('Model A', 2)

In [7]:
model_storage.list_models()

,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTMP,MODEL_STORAGE_VER
0,Model A,1,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""thread_ratio"": null, ""p...",2019-07-28 06:49:48,1


In [6]:
df = model_storage.list_models(name='Model A')

In [ ]:
model_storage.delete_model_batch('Model B')

In [15]:
start_time = '2019-07-27 07:49:48'
end_time = '2019-07-29 10:26:47'
model_storage.delete_model_batch('Model A', start_time=start_time, end_time=end_time)
model_storage.list_models()

mod_name: hana_ml.algorithms.pal.neural_network
mod_name: hana_ml.algorithms.pal.neural_network


,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,MODEL_STORAGE_VER
0,Model A,3,PAL,hana_ml.algorithms.pal.neural_network.MLPRegre...,"{""model_attributes"": {""thread_ratio"": null, ""p...",2019-07-29 10:26:48,1


In [8]:
model_storage.list_models()['JSON'].iloc[0]

'{"model_attributes": {"thread_ratio": null, "param_range": null, "param_values": null, "progress_indicator_id": null, "timeout": null, "random_state": null, "random_search_times": null, "search_strategy": null, "repeat_times": null, "fold_num": null, "evaluation_metric": null, "resampling_method": null, "categorical_variable": null, "weight_init": null, "normalization": null, "batch_size": null, "momentum": 0.001, "learning_rate": 0.01, "training_style": "stochastic", "max_iter": null, "hidden_layer_size_options": null, "hidden_layer_size": [10], "output_activation_options": null, "output_activation": "TANH", "activation_options": null, "activation": "TANH"}, "artifacts": {"schema": "STORAGE", "model_tables": {"model_": "HANAML_MODEL_A_1_CLASS_HANA_ML_ALGORITHMS_PAL_NEURAL_NETWORK_MLPREGRESSOR_MODELS"}, "library": "PAL"}}'

In [9]:

model = model_storage.load_model(name='Model A', version=1)

mod_name: hana_ml.algorithms.pal.neural_network


In [10]:
model.model_.select_statement

'SELECT * FROM "STORAGE"."HANAML_MODEL_A_1_CLASS_HANA_ML_ALGORITHMS_PAL_NEURAL_NETWORK_MLPREGRESSOR_MODELS"'

In [11]:
test = pd.DataFrame()
test['id'] = [1]
test['sepal length (cm)'] = [4.2]
test['sepal width (cm)'] = [3.2]
test['petal length (cm)'] = [1.2]
test['petal width (cm)'] = [0.2]

test_df = create_dataframe_from_pandas(connection_context, test, 'IRIS_TEST', force=True)
model.predict(test_df, key='id').collect()

,id,TARGET,VALUE
0,1,target,0.0
